## A Language Translator

Envs

Setup LangSmith

In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

Language Models

In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
  SystemMessage("Translate the following from English to Nepali"),
  HumanMessage("Hi! how are you doing?")
]

model.invoke(messages)

/Users/thapa24-mbp/Devs/sandbox/ai-agents/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=3c5e887e-e5da-4fbd-9a61-e0b4ce266357,id=3c5e887e-e5da-4fbd-9a61-e0b4ce266357


AIMessage(content='नमस्ते! तपाईलाई कस्तो छ?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--3c5e887e-e5da-4fbd-9a61-e0b4ce266357-0', usage_metadata={'input_tokens': 14, 'output_tokens': 15, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=3c5e887e-e5da-4fbd-9a61-e0b4ce266357,id=3c5e887e-e5da-4fbd-9a61-e0b4ce266357
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=16762d65-6e17-4cdc-9ecd-f6b948e70fdd,id=16762d65-6e17-4cdc-9ecd-f6b948e70fdd; trace=16762d65-6e17-4cdc-9ecd-f6b948e70fdd,id=16762d65-6e17-4cdc-9ecd-f6b948e70fdd
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: U

#### The following are all same:

model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}]) 

// ^--- this is important. it infers that the model.invoke function implementation will take the inputs and pass that to a prompt template. the template has variables like role, and content, which finally will be turned into model ready prompt.

model.invoke([HumanMessage("Hello")])

Runnable interface: Streaming

In [9]:
tokens = model.stream(messages)
for token in tokens:
  print(token.content, end="|")

नमस्ते|! तपाईंलाई कस्तो छ?
|

#### PROMPT TEMPLATE
##### To create a model ready prompt from user input

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages([
  ("system", system_template),
  ("user", "{text}")
])

# (roles, message)
# possible message roles: https://python.langchain.com/docs/concepts/messages/#role

In [12]:
prompt = prompt_template.invoke({"language": "Nepali", "text": "what the hell are you doing?"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Nepali', additional_kwargs={}, response_metadata={}), HumanMessage(content='what the hell are you doing?', additional_kwargs={}, response_metadata={})])

In [13]:
model_result = model.invoke(prompt)
print(model_result.content)

तँ के गरिरहेको छस्? / तिमी के गर्दैछौ? / हजुर के गर्दै हुनुहुन्छ?

Which one you use depends on the level of respect you want to show.

*   **तँ के गरिरहेको छस्?** - Very informal, almost rude. Use with close friends or someone you're angry with.
*   **तिमी के गर्दैछौ?** - Informal, standard way to ask someone you know.
*   **हजुर के गर्दै हुनुहुन्छ?** - Formal, respectful. Use with elders or people you don't know well.

The phrase "what the hell" doesn't directly translate, but the tone of the chosen sentence reflects the level of anger or frustration.


In [15]:
prompt.to_string()

'System: Translate the following from English into Nepali\nHuman: what the hell are you doing?'